
# <center style = 'padding : 20px; border : 3px pink double' >Predicting the returns of orders  for a retail shoe seller</center>

<center><em>Simon DAHAN, Louise LOESCH, Louis SALOMÉ, Martin VERSTRAETE</em></center>



# 1. Introduction
###  Challenge SD210 2018
#### Authors :  Florence D'Alché & Umut Şimşekli & Moussab Djerrab


**Context of the challenge:**

An electronic commerce company sells shoes, and has a high return rate of his products, more than 20%. This large number of returns and exchanges has a negative impact on its margin. To remedy this problem, the company wants to better understand this phenomenon, and have tools to quantify the probability of return for a given product. It makes available its database of orders placed between October 2011 and October 2015, its product feedback data, and its customer and product databases (provide the data dictionary).

**Goal of the challenge:**
<ul>
<li>Identify conditions that favor product return (eg what type of product is usually returned, which customer is more keen on returning a product, what type of order or purchase context most often leads to returns?)</li>
<li>Build a return forecast template for each product from a shopping cart.
</li>
</ul>

To go further: This project aimes at making stand out purchasing behaviors. With this knowledge, the e-merchant wishes to use this data to better plan his activity. In particular, he wants to forecast the turnover generated by his clients.



**Training data:**

There will be $N= 1067290$ lines of orders in the training dataset. For each order  the training dataset reports if the command has been returned (***ReturnQuantityBin***) and the quantity returned (***ReturnQuantity***). The column to target (***ReturnQuantityBin***) which is a binary column ($y = 1$ if returned and $y=0$ otherwise). 

**Test data:**

The test data contain $N_\text{test} = 800468$ lines of orders. Everything else is similar to the training data.


### Additional Data

As part of the challenge, two additional datasets are avalaible namely (**customers.csv**) and (**products.csv**). Those to sets contains informations on custmers and on the products. A good prediction model will necessarily require extraction of information comming from this dataset. Students are free to use these data as they see fit. Please keep in mind that both sets containes also customers and products that are not present in the training or test sets.

A dictionnary of variables (**dictionnary.xlsx**) is avalaible in the folder containing the datasets. Please refer to it so as to have a definition of the variables at hand.


### The goal and the performance criterion

In this challenge, we will use an evaluation metric, which is commonly used in binary prediction, namely the ROC AUC criteria. **The closest to 1 the better (be affarait if its below 0).**
Hence the form of the file to send is of the form :


| <center> probability </center>  |
| ------------- |
| <center> 0.90  </center>         |
| <center> ...  </center>         |
| <center> 0.42  </center>         |


The order of the probabilities needs to respect the order in the test set.



### Training Data

https://www.dropbox.com/sh/uo4oudw43j45mp3/AACA0UqkitNKSWdE_7fs2Wbla?dl=0


In [1]:
from __future__ import division

import os

import numpy as np

import matplotlib
from matplotlib import pyplot as plt

import time
from time import time, mktime

import random

import math

import pandas as pd
from pandas import DataFrame
from datetime import datetime

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score

# 2. Présentation des données

In [26]:
## Lecture des .csv
products = pd.read_csv("products.csv",sep=";", encoding = "ISO-8859-1")
customers = pd.read_csv("customers.csv",sep=",", encoding = "ISO-8859-1")
X_train = pd.read_csv("X_train.csv")
X_test   = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

C:\Users\Martin\workspace\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
## Création des DataFrames
df_train0 = pd.DataFrame(X_train)
df_test0 =pd.DataFrame(X_test)
products0=pd.DataFrame(products)
customers0=pd.DataFrame(customers)

df_train = df_train0.copy()
df_test = df_test0.copy()
products = products0.copy()
customers = customers0.copy()

In [28]:
DETAILS_FEATURES_NAMES = False
N_train = len(df_train)
N_test = len(df_test)
N_products = len(products)
N_customers = len(customers)

#### 2.1 Les commandes

Le Train Set est un ensemble de commandes.

In [29]:
print("Nombre de commandes dans X_train :", df_train.shape[0])
print("Nombre d'information par commande :", df_train.shape[1])

if DETAILS_FEATURES_NAMES == True :
    print("\nDans X_train, les features sont :")
    features_names = list(df_train.columns.values)
    for i in range(len(features_names)):
        print(i+1,". ",features_names[i])

df_train.head(2)

Nombre de commandes dans X_train : 1067290
Nombre d'information par commande : 20


,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,DeviceTypeLabel,PricingTypeLabel,TotalLineItems,Quantity,UnitPMPEUR,OrderCreationDate,OrderShipDate,OrderNumCustomer,IsOnSale,BillingPostalCode
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,"52,64",2011-10-26 12:10:48,2011-10-26 18:27:00,1,0.0,87000
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,"52,64",2011-10-26 12:10:48,2011-10-26 18:27:00,1,0.0,87000


#### Les produits

La table products contient des informations sur les produits.

In [30]:
print("Nombre de produits différents :", products.shape[0])
print("Nombre d'information par produit :", products.shape[1])

if DETAILS_FEATURES_NAMES == True :
    print("\nDans X_train, les features sont :")
    products_features_names = list(products.columns.values)
    for i in range(len(products_features_names)):
        print(i+1,". ",products_features_names[i])

products.head(2)

Nombre de produits différents : 1581839
Nombre d'information par produit : 25


,VariantId,GenderLabel,MarketTargetLabel,SeasonLabel,SeasonalityLabel,BrandId,UniverseLabel,TypeBrand,ProductId,ProductType,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,101,Femme,ND,Automne/Hiver,Saisonnier,255.0,ND,Standard,1.0,Bottes,...,NaN,0.0,28.0,0.0,De pluie,NaN,NaN,NaN,False,Prenez votre pointure habituelle
1,102,Femme,ND,Automne/Hiver,Saisonnier,255.0,ND,Standard,1.0,Bottes,...,NaN,0.0,28.0,0.0,De pluie,NaN,NaN,NaN,False,Prenez votre pointure habituelle


#### Les clients

La table customers contient des informations sur les clients.

In [31]:
print("Nombre de clients différents :", customers.shape[0])
print("Nombre d'information par produit :", customers.shape[1])

if DETAILS_FEATURES_NAMES == True :
    print("\nDans X_train, les features sont :")
    customers_features_names = list(customers.columns.values)
    for i in range(len(customers_features_names)):
        print(i+1,". ",customers_features_names[i])

customers.head(2)

Nombre de clients différents : 7708968
Nombre d'information par produit : 5


,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate
0,10000022,FR,NaN,Homme,2006-02-09 09:52:13
1,10000025,FR,1979-03-17 00:00:00,Homme,2005-09-11 02:04:37


#### GroupBy : Regarder ses données ...

In [32]:
df_train.groupby('CustomerTypeLabel').count()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,IsoCode,DeviceTypeLabel,PricingTypeLabel,TotalLineItems,Quantity,UnitPMPEUR,OrderCreationDate,OrderShipDate,OrderNumCustomer,IsOnSale,BillingPostalCode
CustomerTypeLabel,,,,,,,,,,,,,,,,,,,
Fidélisé,615726,615726,615726,615726,615726,615726,615726,615726,615726,615726,615726,615726,615726,615726,615726,615726,615726,615242,615721
Nouveau,451564,451564,451564,451564,451564,451564,451564,451564,451564,451564,451564,451564,451564,451564,451564,451564,451564,444233,451517


# 3. Sélection des features

On va créer une nouvelle table de features en sélectionnant une à une les features à partir des 3 tables mises à disposition.

## 3.1 Features de la table Order (X_train)

In [33]:
del df_train['BillingPostalCode']

In [34]:
#Conversion pour le train

start = time()
for i in range(N_train):
    
    if df_train['LineItem'][i]>5 : 
        #Si la commande n'est pas dans les 5 premiers, pas de différentiation
        df_train.set_value(i,'LineItem',5)
        
    if df_train['TotalLineItems'][i]>4 : 
        #Si la commande comporte plus de 4 articles, pas de différentiation
        df_train.set_value(i,'TotalLineItems',4)
        
    if df_train['Quantity'][i]>1 : 
        #Si l'article est commandé plus d'une fois, pas de différentiation
        df_train.set_value(i,'Quantity',2)
        
    if df_train['OrderNumCustomer'][i]>3 : 
        #Si l'article est commandé plus d'une fois, pas de différentiation
        df_train.set_value(i,'OrderNumCustomer',3)
        
    if df_train['PayementModeLabel'][i] not in ['Carte bancaire', 'PayPal','Gratuit'] : 
        #On ne garde que 4 catégories de paiement
        df_train.set_value(i,'PayementModeLabel','Autre')
        
    if df_train['IsoCode'][i] != 'FR' :
        df_train.set_value(i,'IsoCode','HorsDeFrance')
    
    # OrderCreationDate devient le mois de commande
    # et OrderShipDate devient la durée entre la commande et l'expédition
    # On pourrait les rename mais on le fera pas déso.
    df_train.set_value(i,'OrderShipDate',
            int(( mktime(pd.Timestamp(df_train['OrderShipDate'][i]).to_pydatetime().timetuple())
                 - mktime(pd.Timestamp(df_train['OrderCreationDate'][i]).to_pydatetime().timetuple()) )/3600 ) ) #en heures
    
    df_train.set_value(i,'OrderCreationDate',pd.Timestamp(df_train['OrderCreationDate'][i]).month)
    
    # Faire des palliers de prix
    df_train.set_value(i,'UnitPMPEUR',df_train['UnitPMPEUR'][i].replace(",","."))
    df_train.set_value(i,'UnitPMPEUR', float(df_train['UnitPMPEUR'][i])//10 + 1) #Faire des palliers de 10euros ?

print("Temps de calcul de la modification des features Train 'Order' :",time()-start)
# ~344 secondes

Temps de calcul de la modification des features Train 'Order' : 331.3799104690552


In [35]:
df_train.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,DeviceTypeLabel,PricingTypeLabel,TotalLineItems,Quantity,UnitPMPEUR,OrderCreationDate,OrderShipDate,OrderNumCustomer,IsOnSale
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,6,10,6,1,0.0
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,6,10,6,1,0.0
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,1,1,2,10,5,1,0.0
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,6,10,5,1,0.0
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,4,10,5,1,0.0


In [36]:
#Conversion pour le test

start = time()
del df_test['BillingPostalCode']

for i in range(N_test):
    
    if df_test['LineItem'][i]>5 : 
        #Si la commande n'est pas dans les 5 premiers, pas de différentiation
        df_test.set_value(i,'LineItem',5)
        
    if df_test['TotalLineItems'][i]>4 : 
        #Si la commande comporte plus de 4 articles, pas de différentiation
        df_test.set_value(i,'TotalLineItems',4)
        
    if df_test['Quantity'][i]>1 : 
        #Si l'article est commandé plus d'une fois, pas de différentiation
        df_test.set_value(i,'Quantity',2)
        
    if df_test['OrderNumCustomer'][i]>3 : 
        #Si l'article est commandé plus d'une fois, pas de différentiation
        df_test.set_value(i,'OrderNumCustomer',3)
        
    if df_test['PayementModeLabel'][i] not in ['Carte bancaire', 'PayPal','Gratuit'] :
        #On ne garde que 4 catégories de paiement
        df_test.set_value(i,'PayementModeLabel','Autre')
        
    if df_test['IsoCode'][i] != 'FR' :
        df_test.set_value(i,'IsoCode','HorsDeFrance')
    
    #OrderCreationDate devient le mois de commande
    # et OrderShipDate devient la durée entre la commande et l'expédition
    # On pourrait les rename mais on le fera pas déso.
    df_test.set_value(i,'OrderShipDate',
            int(( mktime(pd.Timestamp(df_test['OrderShipDate'][i]).to_pydatetime().timetuple())
                 - mktime(pd.Timestamp(df_test['OrderCreationDate'][i]).to_pydatetime().timetuple()) )/3600 ) ) #en heures
    
    df_test.set_value(i,'OrderCreationDate',pd.Timestamp(df_test['OrderCreationDate'][i]).month)
    
    # Faire des palliers de prix
    df_test.set_value(i,'UnitPMPEUR',df_test['UnitPMPEUR'][i].replace(",","."))

print("Temps de calcul de la modification des features Test 'Customer' :",time()-start)
# ~144 secondes

Temps de calcul de la modification des features Test 'Customer' : 126.63765239715576


In [37]:
df_test.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,DeviceTypeLabel,PricingTypeLabel,TotalLineItems,Quantity,UnitPMPEUR,OrderCreationDate,OrderShipDate,OrderNumCustomer,IsOnSale
0,90607166,754279,1,14369241,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Fidélisé,HorsDeFrance,Desktop,Promo Avec CP,2,1,52.92,12,28,3,0.0
1,90621016,1043714,1,16121040,Expédié,DIRECT,Automne/Hiver,Autre,Fidélisé,HorsDeFrance,Desktop,Promo Avec CP,1,1,32.4355,12,20,2,0.0
2,90630628,302486,1,16271310,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,Desktop,Plein Tarif,1,1,66.65,12,26,1,0.0
3,90641377,1300698,1,16271314,Expédié,DIRECT,Automne/Hiver,PayPal,Nouveau,HorsDeFrance,Desktop,Promo Avec CP,1,1,35.72,12,22,1,0.0
4,90652456,1392714,1,16251703,Expédié,DIRECT,Automne/Hiver,Autre,Nouveau,HorsDeFrance,Desktop,Plein Tarif,1,1,23.86,12,50,1,0.0


## 3.2 Features de la table Products

On commence par merge les 3 tables

In [38]:
print("Les featres de customers sont : \n",products.columns.values)

Les featres de customers sont : 
 ['VariantId' 'GenderLabel' 'MarketTargetLabel' 'SeasonLabel'
 'SeasonalityLabel' 'BrandId' 'UniverseLabel' 'TypeBrand' 'ProductId'
 'ProductType' 'SupplierColor' 'ProductColorId' 'MinSize' 'MaxSize'
 'CalfTurn' 'UpperHeight' 'HeelHeight' 'PurchasePriceHT' 'IsNewCollection'
 'SubtypeLabel' 'UpperMaterialLabel' 'LiningMaterialLabel'
 'OutSoleMaterialLabel' 'RemovableSole' 'SizeAdviceDescription']


In [39]:
start=time()

for i in range (N_products):
    ### Louise
    if products['ProductType'][i] not in ['Chaussures à lacets','Ballerines','Baskets','Bottines et boots','Escarpins','Sandales et nu-pieds']:
        products.set_value(i,'ProductType','Autre')
        
    # On classe les conseils en catégories
    if (type(products['SizeAdviceDescription'][i])!=str):
        products.set_value(i,'SizeAdviceDescription','Chaussure confortable, pointure habituelle');
    else:
        if (products['SizeAdviceDescription'][i].find('dessous')>0):
            products.set_value(i,'SizeAdviceDescription','Prenez une taille en dessous de votre pointure habituelle')
        elif (products['SizeAdviceDescription'][i].find('dessus')>0): 
            products.set_value(i,'SizeAdviceDescription','Prenez une taille au-dessus de votre pointure habituelle')
        else: 
            products.set_value(i,'SizeAdviceDescription','Chaussure confortable, pointure habituelle');
    
    ### Simon
    if products['MarketTargetLabel'][i] not in ['Contemporaine', 'Classique','Fashionistas',
                                                'ND','Confort','Suiveur',np.NaN] : 
            #On ne garde que 5 catégories cibles de marché
            products.set_value(i,'MarketTargetLabel','Autre')
            
    if products['BrandId'][i] not in [769.0,62.0,154.0,439.0,111.0]: 
            #On ne garde que 5 catégories cibles de marché
            products.set_value(i,'BrandId',np.NaN)
            
    if products['UniverseLabel'][i] not in ['Détente', 'Green','Luxe',
                                                'ND','Sport','Ville',np.NaN] : 
            #On ne garde que 7 catégories cibles de marché
            products.set_value(i,'UniverseLabel','Autre') 
            
    ### Martin
    if products['SubtypeLabel'][i] not in ['Basses','Bout rond','A boucles','A lacets','A scratch / velcro','Montantes','Autres']:
        products.set_value(i,'SubtypeLabel','Autres')
        
        
    if (math.isnan(products['CalfTurn'][i])):
        products.set_value(i,'CalfTurn',np.nan)
    if (products['CalfTurn'][i]<=40 and products['CalfTurn'][i]>=34): 
        products.set_value(i,'CalfTurn',37)
    if (products['CalfTurn'][i]==0): 
        products.set_value(i,'CalfTurn',0)
    if (products['CalfTurn'][i]>0 and products['CalfTurn'][i]<34): 
        products.set_value(i,'CalfTurn',17)
    if (products['CalfTurn'][i]>40): 
        products.set_value(i,'CalfTurn',52)
    if products['UpperMaterialLabel'][i] not in ['Cuir','Nubuck']:
        products.set_value(i,'UpperMaterialLabel','Autre')
    if products['LiningMaterialLabel'][i] not in ['Cuir','Textile','CuirTextile','Synthétique']:
        products.set_value(i,'LiningMaterialLabel','Autre')
    if products['OutSoleMaterialLabel'][i] not in ['Caoutchouc','Gomme','Synthétique']:
        products.set_value(i,'OutSoleMaterialLabel','Autre')

print("Temps de calcul de la modification des features Test 'Products' :",time()-start)

Temps de calcul de la modification des features Test 'Products' : 632.9708528518677


In [40]:
products.head(10)

,VariantId,GenderLabel,MarketTargetLabel,SeasonLabel,SeasonalityLabel,BrandId,UniverseLabel,TypeBrand,ProductId,ProductType,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,101,Femme,ND,Automne/Hiver,Saisonnier,NaN,ND,Standard,1.0,Autre,...,NaN,0.0,28.0,0.0,Autres,Autre,Autre,Autre,False,"Chaussure confortable, pointure habituelle"
1,102,Femme,ND,Automne/Hiver,Saisonnier,NaN,ND,Standard,1.0,Autre,...,NaN,0.0,28.0,0.0,Autres,Autre,Autre,Autre,False,"Chaussure confortable, pointure habituelle"
2,103,Femme,ND,Automne/Hiver,Saisonnier,NaN,ND,Standard,1.0,Autre,...,NaN,0.0,28.0,0.0,Autres,Autre,Autre,Autre,False,"Chaussure confortable, pointure habituelle"
3,104,Femme,ND,Automne/Hiver,Saisonnier,NaN,ND,Standard,1.0,Autre,...,NaN,0.0,28.0,0.0,Autres,Autre,Autre,Autre,False,"Chaussure confortable, pointure habituelle"
4,105,Femme,ND,Automne/Hiver,Saisonnier,NaN,ND,Standard,1.0,Autre,...,NaN,0.0,28.0,0.0,Autres,Autre,Autre,Autre,False,"Chaussure confortable, pointure habituelle"
5,106,Femme,ND,Automne/Hiver,Saisonnier,NaN,ND,Standard,1.0,Autre,...,NaN,0.0,28.0,0.0,Autres,Autre,Autre,Autre,False,"Chaussure confortable, pointure habituelle"
6,107,Femme,ND,Automne/Hiver,Saisonnier,NaN,ND,Standard,1.0,Autre,...,NaN,0.0,28.0,0.0,Autres,Autre,Autre,Autre,False,"Chaussure confortable, pointure habituelle"
7,108,Femme,ND,Automne/Hiver,Saisonnier,NaN,ND,Standard,1.0,Autre,...,NaN,0.0,28.0,0.0,Autres,Autre,Autre,Autre,False,"Chaussure confortable, pointure habituelle"
8,109,Femme,ND,Automne/Hiver,Saisonnier,NaN,ND,Standard,1.0,Autre,...,NaN,0.0,28.0,0.0,Autres,Autre,Autre,Autre,False,"Chaussure confortable, pointure habituelle"
9,110,Femme,ND,Automne/Hiver,Saisonnier,NaN,ND,Standard,1.0,Autre,...,NaN,0.0,28.0,0.0,Autres,Autre,Autre,Autre,False,"Chaussure confortable, pointure habituelle"


## 3.3 Features de la table Customers

In [41]:
print("Les featres de customers sont : \n",customers.columns.values)

Les featres de customers sont : 
 ['CustomerId' 'CountryISOCode' 'BirthDate' 'Gender' 'FirstOrderDate']


In [42]:
del customers['FirstOrderDate']

In [43]:
start = time()

for i in range(N_customers) :
    # Cette feature donne le nombre de jour entre la première commande et cette nouvelle commande
    
    #Cette feature donne l'âge (2018 - année de naissance)
    if (type(customers['BirthDate'][i])!=str):
        if (math.isnan(customers['BirthDate'][i])):
            continue
    else:
        if (int(customers['BirthDate'][i][0:4])>2018):
            customers.set_value(i,'BirthDate',np.NaN)
            continue
        else:
            customers.set_value(i,'BirthDate',2018-int(customers['BirthDate'][i][0:4]))
            
    if customers['CountryISOCode'][i] != 'FR' :
    # Soit en Frace, soit à l'étranger
        customers.set_value(i,'CountryISOCode','HorsDeFrance')

print("Temps de calcul de la modification des features Test 'Order' :",time()-start)
# ~756 secondes

Temps de calcul de la modification des features Test 'Order' : 637.9142825603485


In [44]:
customers.head(10)

,CustomerId,CountryISOCode,BirthDate,Gender
0,10000022,FR,NaN,Homme
1,10000025,FR,39,Homme
2,10000031,FR,NaN,Homme
3,10000046,FR,NaN,Homme
4,10000047,FR,34,Femme
5,10000059,FR,44,Homme
6,10000069,FR,NaN,Femme
7,10000080,FR,58,Homme
8,10000102,FR,53,Femme
9,10000103,FR,36,Femme


## 3.4 Merge

In [45]:
# Merge pour Train
result = pd.merge(df_train, customers, on='CustomerId',how='left')
df_train = pd.merge(result, products, on='VariantId',how='left')

df_test['Ordre_proba']=pd.Series(df_test.index)

# Merge pour test
df_test = pd.merge(df_test, customers, on='CustomerId',how='left')
df_test = pd.merge(df_test, products, on='VariantId',how='left')

# Order à mémoriser pour les tests
df_test=df_test.sort_values(['Ordre_proba'])

In [46]:
print("Il y a ",df_train.shape[1],"features.")
print(df_train.shape)
print(df_test.shape)
df_train.head()

Il y a  46 features.
(1067290, 46)
(800468, 47)


,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel_x,PayementModeLabel,CustomerTypeLabel,IsoCode,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,12.0,8.0,15.5,0.0,Bout rond,Autre,Autre,Autre,False,"Chaussure confortable, pointure habituelle"
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,5.0,9.0,37.0,0.0,Bout rond,Cuir,Autre,Autre,False,"Chaussure confortable, pointure habituelle"


# 4. Classification

On s'assure que le type des colonnes créées est bien différent d'objet pour optimiser le mask par la suite

In [48]:
df_test['PurchasePriceHT']=df_test['PurchasePriceHT'].astype('float32')
df_train['PurchasePriceHT']=df_train['PurchasePriceHT'].astype('float32')
df_train['BirthDate']=df_train['BirthDate'].astype('float32')
#df_test['FirstOrderDate']=df_test['FirstOrderDate'].astype('float32')
df_test['BirthDate']=df_test['BirthDate'].astype('float32')
#df_train['FirstOrderDate']=df_train['FirstOrderDate'].astype('float32')
df_test['OrderCreationDate']=df_test['OrderCreationDate'].astype('float32')
df_train['OrderCreationDate']=df_train['OrderCreationDate'].astype('float32')
df_train['UnitPMPEUR']=df_train['UnitPMPEUR'].astype('float32')
df_test['UnitPMPEUR']=df_test['UnitPMPEUR'].astype('float32')
df_test['OrderShipDate']=df_test['OrderShipDate'].astype('float32')
df_train['OrderShipDate']=df_train['OrderShipDate'].astype('float32')
df_test['PurchasePriceHT']=df_test['PurchasePriceHT'].astype('float32')
df_train['PurchasePriceHT']=df_train['PurchasePriceHT'].astype('float32')

#### On scale si besoin (déconseillé)

In [49]:
min_max_scaler = preprocessing.MinMaxScaler()
SCALE=False

if SCALE==True:
    df_train['OrderCreationDate'] = pd.DataFrame(min_max_scaler.fit_transform(pd.DataFrame(df_train['OrderCreationDate'])),columns=['OrderCreationDate'])
    df_test['OrderCreationDate'] = pd.DataFrame(min_max_scaler.fit_transform(pd.DataFrame(df_test['OrderCreationDate'])),columns=['OrderCreationDate'])
    df_train['OrderShipDate'] = pd.DataFrame(min_max_scaler.fit_transform(pd.DataFrame(df_train['OrderShipDate'])),columns=['OrderShipDate'])
    df_test['OrderShipDate'] = pd.DataFrame(min_max_scaler.fit_transform(pd.DataFrame(df_test['OrderShipDate'])),columns=['OrderShipDate'])

On sauvegarde les nouveaux datasets

In [50]:
df_train.to_csv("df_train.csv",sep=";",encoding="utf-8")
df_test.to_csv("df_test.csv",sep=";",encoding ="utf-8")

On load les données transformées

In [2]:
newX_train = pd.read_csv("df_train.csv",sep=";",encoding ="utf-8")
newX_test   = pd.read_csv("df_test.csv",sep=";",encoding ="utf-8")

In [3]:
y_train = pd.read_csv("y_train.csv")

In [4]:
df_train = pd.DataFrame(newX_train)
df_test=pd.DataFrame(newX_test)

# On drop la colonne inutile
df_train = df_train.drop('Unnamed: 0', 1)
df_test = df_test.drop('Unnamed: 0', 1)

# On libère de la mémoire
newX_train = 0
newX_test = 0

## Defining a feature transformation

In [24]:
df_train.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel_x,PayementModeLabel,CustomerTypeLabel,IsoCode,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,12.0,8.0,15.5,0.0,Bout rond,Autre,Autre,Autre,False,"Chaussure confortable, pointure habituelle"
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,5.0,9.0,37.0,0.0,Bout rond,Cuir,Autre,Autre,False,"Chaussure confortable, pointure habituelle"


In [17]:
def funk_mask1(d):
    " Defining a simple mask over the input data "
    columns_ext = ["VariantId","ProductId","MinSize","MaxSize","SupplierColor","OrderShipDate","ProductColorId",
                   "SubtypeLabel","Ordre_proba","OrderStatusLabel",
                   "OrderNumber","VariantId", "CustomerId","BillingPostalCode",
                   "SizeAdviceDescription","OrderCreationDate","PurchasePriceHT"]
    X1 = d.loc[:,[xx for xx in d.columns if xx not in columns_ext]]
    columns2bin = [x for x in X1.columns if X1[x].dtype == np.dtype('O')]
    X2 = pd.get_dummies(X1.loc[:,columns2bin])
    X1 = X1.loc[:,[xx for xx in X1.columns if xx not in columns2bin]]
    res = pd.concat([X1,X2],axis=1)
    res = res.fillna(0)
    return(res)

Il y a quelques features qui posent problème pour l'instant, donc je sépare les colonnes en deux et je crée une autre fonction qui gère la deuxième partie.

## Applying the mask

In [18]:
x1 = funk_mask1(df_train)
x2= funk_mask1(df_test)
seleckt_columns = np.intersect1d(x1.columns,x2.columns)
x1 = x1.loc[:,seleckt_columns]
x2 = x2.loc[:,seleckt_columns]

## Supervised learning : Random Forest

In [22]:
debut=time()
clf = RandomForestClassifier(n_estimators=200,max_features = 15,criterion = 'entropy', max_depth = 20, min_samples_leaf = 1000)
clf.fit(x1, y_train.ReturnQuantityBin)
y_tosubmit = clf.predict_proba(x2.loc[:,x1.columns]) ## Fichier à submit, allez directement à la dernière case
print(time()-debut)

511.75302839279175


<em style = "color : red">ATTENTION : la case qui suit contient une cv en 8 parties, donc 8 fois plus longue à exécuter que la case précédente</em>

In [ ]:
#predictions = clf.predict(X)
debut=time()
score = cross_val_score(clf,x1,y_train.ReturnQuantityBin, cv=8).mean()
print("Score obtenu :",score)
print(time()-debut)

In [ ]:
debut=time()
scores=[]
for max_features in [13,14]:
    clf = RandomForestClassifier(n_estimators=100,max_features = max_features,criterion = 'entropy', max_depth = 15, min_samples_leaf = 20)
    clf.fit(x1, y_train.ReturnQuantityBin)
    y_tosubmit = clf.predict_proba(x2.loc[:,x1.columns])
    score = cross_val_score(clf,x1,y_train.ReturnQuantityBin, cv=8).mean()
    scores.append(score)

plt.figure()
plt.title("Score de la régression en fonction du nombre max de features")
plt.plot([13,14],scores)
print(time()-debut)
plt.show()

In [ ]:
debut=time()
param_grid = [ {'max_depth': [i for i in range(10,25)] , 'min_samples_leaf': [i for i in range(10,50)]} ]
clf = sklearn.model_selection.GridSearchCV(RandomForestClassifier(),
                                     param_grid,
                                           cv=5,
                                     scoring= 'accuracy')
clf.fit(x1,y_train.ReturnQuantityBin)
print("Best parameters set found on development set:\n",clf.best_params_)
print(time()-debut)

## Score of our prediction : on the train

In [23]:
yres = clf.predict_proba(x1.loc[:100000,x1.columns])
roc_auc_score(y_train.ReturnQuantityBin.iloc[:100001],yres[:,1])

# Submission to the system
np.savetxt('y_pred.txt', y_tosubmit[:,1], fmt='%f')